In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Obstructive_sleep_apnea/GSE133601'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional survey of peripheral blood links lower oxygen saturation during sleep with reduced expressions of CD1D and RAB20 that is reversed by CPAP therapy"
!Series_summary	"Sleep Disordered Breathing (SDB) is associated with a wide range of physiological changes, likely due in part to the influence of hypoxemia during sleep on gene expression. We studied gene expression in peripheral blood mononuclear cells in association with three measures of SDB: the Apnea Hypopnea Index (AHI); average oxyhemoglobin saturation (avgO2) during sleep; and minimum oxyhemoglobin saturation (minO2) during sleep. We performed discovery analysis in two community-based studies: the Multi-Ethnic Study of Atherosclerosis (MESA; N = 580) and the Framingham Offspring Study (FOS; N=571). Associations with false discovery rate (FDR) q-value<0.05 in one study were considered to have replicated if a p-value<0.05 was observed in the other study. Associations that replicat

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
background_gene_info = "!Series_title\t\"Transcriptional survey of peripheral blood links lower oxygen saturation during sleep with reduced expressions of CD1D and RAB20 that is reversed by CPAP therapy\""
if "Transcriptional" in background_gene_info:
    is_gene_available = True

# Determine variable availability and conversion functions
sample_characteristics = {
    0: ['tissue: peripheral blood mononuclear cells'], 
    1: ['subject: 10031', 'subject: 11874', 'subject: 11992', 'subject: 30234', 'subject: 30665', 'subject: 30838', 'subject: 40044', 'subject: 40266', 'subject: 40657', 'subject: 11928', 'subject: 30031', 'subject: 40269', 'subject: 30624', 'subject: 40971', 'subject: 40197'], 
    2: ['timepoint: pre-CPAP', 'timepoint: post-CPAP']
}

# Evaluating data availability based on explicit and inferred information
# 'Obstructive_sleep_apnea' trait
if 2 in sample_characteristics:
    unique_values = set([entry.split(":")[1].strip() for entry in sample_characteristics[2]])
    if len(unique_values) > 1:
        trait_row = 2

# Function to convert 'Obstructive_sleep_apnea' values
def convert_trait(value):
    value = value.split(":")[1].strip().lower()
    if value == "pre-cpap":
        return 0
    elif value == "post-cpap":
        return 1
    return None

# Add heuristic checks for age and gender
for key, values in sample_characteristics.items():
    if age_row is None:
        for value in values:
            if "age" in value.lower():
                age_row = key
                break
    if gender_row is None:
        for value in values:
            if "gender" in value.lower() or "sex" in value.lower():
                gender_row = key
                break

# As no explicit info on 'age' and 'gender' is found, set their functions to None
convert_age = None
convert_gender = None

# Save cohort information
save_cohort_info('GSE133601', './preprocessed/Obstructive_sleep_apnea/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction (if applicable)
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Obstructive_sleep_apnea', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Obstructive_sleep_apnea/trait_data/GSE133601.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3912810': [0], 'GSM3912811': [1], 'GSM3912812': [0], 'GSM3912813': [1], 'GSM3912814': [0], 'GSM3912815': [1], 'GSM3912816': [0], 'GSM3912817': [1], 'GSM3912818': [0], 'GSM3912819': [1], 'GSM3912820': [0], 'GSM3912821': [1], 'GSM3912822': [0], 'GSM3912823': [1], 'GSM3912824': [0], 'GSM3912825': [1], 'GSM3912826': [0], 'GSM3912827': [1], 'GSM3912828': [0], 'GSM3912829': [1], 'GSM3912830': [0], 'GSM3912831': [1], 'GSM3912832': [0], 'GSM3912833': [1], 'GSM3912834': [0], 'GSM3912835': [1], 'GSM3912836': [0], 'GSM3912837': [1], 'GSM3912838': [0], 'GSM3912839': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009676_at', '10000_at', '10001_at', '10002_at', '100033413_at',
       '100033414_at', '100033416_at', '100033418_at', '100033420_at',
       '100033422_at', '100033423_at', '100033424_at', '100033425_at',
       '100033426_at', '100033427_at', '100033428_at', '100033430_at',
       '100033431_at', '100033432_at', '100033433_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'SPOT_ID': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the identifier and gene symbol keys from the gene annotation preview.
identifier_key = 'ID'
gene_symbol_key = 'Description'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Obstructive_sleep_apnea/gene_data/GSE133601.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Obstructive_sleep_apnea')

# Define trait_biased by assigning the value of trait_biased
trait_biased = trait_biased

# 4. Save the cohort information.
save_cohort_info('GSE133601', './preprocessed/Obstructive_sleep_apnea/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Obstructive_sleep_apnea/GSE133601.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Obstructive_sleep_apnea', the least common label is '0.0' with 15 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Obstructive_sleep_apnea' in this dataset is fine.

